In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import scipy.sparse
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm

In [2]:
ai_patents = pd.read_csv('../../../ImpactofAI/data/ai_patents_keywrods_alicpc_07092024.csv')

In [3]:
location_table = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/raw/patentsview/g_location_disambiguated.tsv', on_bad_lines = 'skip', sep = '\t')

In [4]:
assignee_table = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/raw/patentsview/g_assignee_disambiguated.tsv', on_bad_lines= 'skip', sep ='\t')



In [37]:
set(assignee_table['type'])

{'reissue', 'utility'}

In [6]:
assignee_table = assignee_table .merge(ai_patents[['patent_id', 'type', 'number', 'country', 'date', 'year', 'abstract','title', 'kind','paper_id','citations', 'references',
       'disruption', 'edm_disruption' ]], on ='patent_id')

In [40]:
assignee_table['country']

0       US
1       US
2       US
3       US
4       US
        ..
3179    US
3180    US
3181    US
3182    US
3183    US
Name: country, Length: 3184, dtype: object

In [7]:
disruptive_threshold = assignee_table['disruption'].quantile(0.75)
consolidating_threshold = assignee_table['disruption'].quantile(0.25)

In [ ]:
 assignee_table['binary_disruptiveness(patent)'] = assignee_table['disruption'].apply(lambda x: 1 if x>disruptive_threshold else 0)

In [ ]:
 assignee_table['binary_consolidating(patent)'] =  assignee_table['disruption'].apply(lambda x: 1 if x<consolidating_threshold else 0)

In [41]:
assignee_table[['patent_id', 'type', 'number', 'year', 'country','kind','title','assignee_id', 'disambig_assignee_organization','disruption','binary_disruptiveness(patent)','binary_consolidating(patent)']].to_csv('../../data/patent_data_to_share.csv',index=False)



In [11]:
df_county = pd.read_csv('../../data/county_naics_2022.txt', dtype={'industry_code': str, 'area_fips': str})
df_county['sector_code'] = df_county['industry_code'].str[:2]
df_county['fips'] = df_county['area_fips'].str[:2]
df_county.head()

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,...,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg,sector_code,fips
0,01000,0,10,50,0,2022,A,NaN,148979,2026102,...,841800853,5.1,-48478330,-18.1,57,5.5,2951,5.4,10,01
1,01000,1,10,51,0,2022,A,NaN,1257,55220,...,0,0.0,0,0.0,52,3.0,2691,3.0,10,01
2,01000,1,102,52,0,2022,A,NaN,1257,55220,...,0,0.0,0,0.0,52,3.0,2691,3.0,10,01
3,01000,1,1021,53,0,2022,A,NaN,619,11505,...,0,0.0,0,0.0,42,3.1,2172,3.1,10,01
4,01000,1,1022,53,0,2022,A,NaN,2,11,...,0,0.0,0,0.0,19,3.0,978,3.0,10,01


In [12]:

msa_shapes = gpd.read_file("/data/sg/munjkim/USPTO/nokia_intern/data/raw/tl_2023_us_cbsa.shp")
msa_shapes = msa_shapes.to_crs("EPSG:4326")  



ERROR 1: PROJ: proj_create_from_database: Open of /home/munjkim/bin/envs/llm/share/proj failed


In [15]:


usjobs = pd.read_excel("../../data/MSA_M2022_dl.xlsx")  # ← replace with your path



In [17]:
msa_conus = msa_shapes[
    ~msa_shapes["NAME"].str.contains(", AK|, HI", regex=True)
]

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,17.7,26.44,37.86,21640,27440,36820,55000,78740,NaN,NaN
1,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,37.36,52.92,76.2,37140,53760,77720,110070,158500,NaN,NaN
2,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,...,35.15,52.3,77.42,30720,46640,73120,108780,161030,NaN,NaN
3,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2021,Marketing Managers,...,44.25,62.83,88.94,55450,66240,92040,130680,185000,NaN,NaN
4,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2022,Sales Managers,...,40.4,61.07,76.85,45380,54090,84020,127030,159850,NaN,NaN


In [33]:
# Replace "**" with NaN, then convert to numeric
usjobs["TOT_EMP"] = pd.to_numeric(usjobs["TOT_EMP"].replace("**", pd.NA), errors="coerce")

# Now group and aggregate
msa_summary = usjobs.groupby(["AREA", "AREA_TITLE"], as_index=False).agg(
    total_workers=("TOT_EMP", "sum"),          # sum of available jobs
    occupations_count=("TOT_EMP", "count"),    # number of occupations with valid data
    missing_jobs=("TOT_EMP", lambda x: x.isna().sum())  # how many were suppressed
)

print(msa_summary.head())


    AREA             AREA_TITLE  total_workers  occupations_count  \
0  10180            Abilene, TX       205110.0                291   
1  10380  Aguadilla-Isabela, PR       144330.0                205   
2  10420              Akron, OH       930890.0                487   
3  10500             Albany, GA       169450.0                256   
4  10540             Albany, OR       135630.0                256   

   missing_jobs  
0             0  
1             2  
2             5  
3             1  
4             2  


In [36]:
msa_summary.to_csv('../../data/msa_data_to_share_09282025.csv',index=False)

In [22]:
usjobs[['AREA', 'AREA_TITLE',"TOT_EMP"]]

,AREA,AREA_TITLE,TOT_EMP
0,10180,"Abilene, TX",70730
1,10180,"Abilene, TX",4470
2,10180,"Abilene, TX",2290
3,10180,"Abilene, TX",50
4,10180,"Abilene, TX",260
...,...,...,...
148199,79600,"Worcester, MA-CT",6380
148200,79600,"Worcester, MA-CT",290
148201,79600,"Worcester, MA-CT",1470
148202,79600,"Worcester, MA-CT",5390


In [17]:
disruptive_tasks = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/processed/disruptive_tasks.csv', on_bad_lines= 'skip')
consolidating_tasks = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/processed/consolidating_tasks.csv', on_bad_lines= 'skip')


In [19]:
consolidating_tasks.columns

Index(['Unnamed: 0', 'O*NET-SOC Code', 'Task ID', 'Scale ID', 'Category',
       'Data Value', 'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound',
       'Recommend Suppress', 'Date', 'Domain Source', 'Task',
       'patents_cosine_similarity', 'ai_patents_matched_id', 'patent_id',
       'type', 'number', 'country', 'date', 'year', 'abstract', 'title',
       'kind', 'paper_id', 'citations', 'references', 'disruption',
       'edm_disruption'],
      dtype='object')

In [ ]:
tasks_merged_df = pd.merge(tasks_merge, location_table, on='location_id', how='left')



disruptive_merged_df = pd.merge(disruptive_merge, location_table, on='location_id', how='left')


consolidating_merged_df = pd.merge(consolidating_merge, location_table, on='location_id', how='left')


In [4]:
task_ratings = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/raw/ONET_ALI/db_24_3_text/Task Ratings.txt', sep='\t').query('`Scale ID` == "IM"')
task_statements = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/raw/ONET_ALI/db_24_3_text/Task Statements.txt', sep='\t')
jobs = pd.read_csv('/data/sg/munjkim/USPTO/nokia_intern/data/raw/ONET_ALI/db_24_3_text/Occupation Data.txt', sep='\t')
jobs['SOC Code'] = jobs['O*NET-SOC Code'].str[:7]
tasks = pd.merge(
    task_ratings,
    task_statements[['O*NET-SOC Code', 'Task ID', 'Task']],
    how='left',
    on=['O*NET-SOC Code', 'Task ID']
)

In [5]:
tasks = pd.merge(
    task_ratings,
    task_statements[['O*NET-SOC Code', 'Task ID', 'Task']],
    how='left',
    on=['O*NET-SOC Code', 'Task ID']
)

In [6]:
ai_patent_both_methods_embedding_id_patent_id_dict = ai_patents.groupby('embedding_id')['patent_id'].apply(list).to_dict()

In [7]:
ai_embedding_id = list(ai_patents['embedding_id'])

In [8]:
ai_embedding_id_iloc_id = {iloc_id:i for iloc_id, i in enumerate(ai_embedding_id)}

In [9]:
tasks['patent_id'] = tasks['ai_patents_matched_id'].apply(lambda x: ai_patent_both_methods_embedding_id_patent_id_dict[ai_embedding_id_iloc_id[x]][0])

KeyError: 'ai_patents_matched_id'